# Multithreading, Multiprocessing und (AsyncIO) Co-Routines
## und was macht da eigentlich das Betriebsystem?!?


# Motivation

- Betriebssysteme auf der TU
- Long time ago: ein multi-cloud client
 - damals mit thread pools
- Performance?
  - Moores Law in Kernen(Transistoren) nicht in einzel CPU Power
  - Skalierbarkeit (z.B. viele Netzwerkverbindungen)
  

# Begriffe

- Nebenläufigkeit / Concurrency
- Paralellisierung / Paralelissm


<img src="https://files.realpython.com/media/Screen_Shot_2018-10-17_at_3.18.44_PM.c02792872031.jpg" >



# CPU und Multitasking/processing?

- Prozesse koennen nur auf den eigenen Speicher zugreifen
    - Pagetable/Speichervirtualisierung
- Sicherheitsmodell (Ringe)
    -Kernelspace hat mehr prvilegien als Userspace
    - Bestimmte Instruktionen und Register(Parameter) koennen gesperrt werden (nur mehr im Kernelspace)


# Threads, Prozesse, Userspace Threads/Coroutines
 - Sind Wege um Nebenläufigkeit zu verwenden
     
 - Scheduler im OS
     - Threads 
     - Prozesse
 - Scheduler im Userspace:
     - Coroutinen -> forsetzbare funktionen

# Prozesse
 - Jeder Prozess sein eigener Heap
 - Prozessattribute
     - Umgebungsvariablen
     - PID
     - Prio etc.
 - Eigenen Virtuellen Speicherbereich
 - Eigener Stack
 ```c
 void bla() {
     int a=1; // <-- stack
 ```
 - Eigener Heap
 ```c
int* xyc = malloc(1024); // <-- heap
 ```
 - Wenn Datenaustausch notwendig, dann Aufwendiger (shared memory)
 - Betriebssystem wechselt zwischen Prozessen

    

## Kernel Threads
- Pro Pozess -> N Threads
- Leichtgewichtiger als Prozess
- Geteilter Heap Speicher
- Eigener Stack
- Jeder Thread sein eigener Stack
- OS-Level Context Switching


## Wechseln zwischen Threads und Prozessen
 1. Ein Interrupt oder Syscall(zB. lesen eines Files) wechselt in den Kernel
 1. Der aktuelle CPU-State wird gespeichert
 1. Der Kernel State wieder geladen
 1. Ein neuer Prozess/Thread wird vom Scheduler ausgesucht
 1. Der Kernel State wierd wieder gespeichert
 1. Der CPU state wird wieder geladen
 1. Wechsel in den Userspace
 
 <img src="https://www.ibm.com/support/knowledgecenter/en/SSGU8G_12.1.0/com.ibm.admin.doc/admin013.gif">


 # Context switching Performance
- Benoetigt Zeit [3] (etwa 1-2 us)
 - Eine Python iteration braucht (~15ns) `1+1`
 - Wichtig? JA! (Aber es kommt drauf an...)
 - 100.000 Gleichzeitige Threads... 0.1-0.2s!
- Unterschiedlicher Speicher
 - Verursacht cache misses
- Worst case es wechselt der CPU Kern, dann kann gar kein Cache mehr verwendet werden.

<img src="https://www.adamh.cz/public/img/context-switch-on-the-arm-cortex-m0/os_intro.png">
 

# Co-Routines
 - Alternative: der User Prozess entscheided wann er unterbrochen werden kann -> in Python Generatoren
 ```py
 def abc_generator():
    yield("a")
    yield("b")
    yield("c")
 ```
 - Funktionen die unterbrochen werden koennen
 - ... und dann wieder weiterlaufen koennen.
 - Vermutlich troztdem Cache misses (Vermutlich weniger)

# Was heisst das jetzt in Python?
 - `threading` -> Nebenlaeufkeit aber keine Paralellisierung (GIL)
 - `multiprocessing` -> Nebenlaeufigkeit und Paralellisierung
 - `co-routines` -> Userspace "threads" -> `asyncio`, `greenlet`, `twisted` etc..

# Multithreading
- wir teilen einen Heap
- ein Stack pro Thread
- Gut fuer einfache probleme mit *Nebenlaeufigkeit*
- Schlecht wenn Multiprozessorsystem genutzt werden wollen
- context switches

# Multiprocessing
- Wir haben einen Heap pro Prozess
- Gil kein Thema
- Inter-Prozess-Kommunikation (IPC) is langsam
- Kein geteilter Speicher, alles muss pickelbar sein
- kontext wechsel

# Co-Routinen
- Ungewohnt
- Micro-threading
- Weniger kontext wechsel
- Fast kein locking!

## GIL
 - Eine VM-Instruktion gleichzeitig
 - Vorteil: teilweise atomar
 - Nachteil: Multiprocessorsysteme haben keinen Vorteil in einer Python Instanz

In [ ]:
# A try for a Benchmark on the python side

In [7]:
# How much is a lock?

from timeit import timeit
import threading
import multiprocessing

def local():
    lock = True
    for _ in range(100):
        lock = False
        lock = True

def thread():
    lock = threading.Lock()
    for _ in range(100):
        lock.acquire()
        lock.release()

def mp():
    lock = multiprocessing.Lock()
    for _ in range(100):
        lock.acquire()
        lock.release()

number = 100000
print('local', timeit(local, number=number))
print('threading', timeit(thread, number=number))
print('mp', timeit(mp, number=number))

local 0.22460610099915357
threading 1.4975726880002185
mp 6.013863459998902


In [1]:
from timeit import timeit
import threading
import multiprocessing

def target():
    pass

def local():
    for _ in range(100):
        target()

def thread():
    for _ in range(100):
        threading.Thread(target=target).start()

def mp():
    for _ in range(100):
        multiprocessing.Process(target=target).start()

number = 100
print('local', timeit(local, number=number))
print('threading', timeit(thread, number=number))
print('mp', timeit(mp, number=number))


local 0.0005302349964040332
threading 0.5513019259960856
mp 23.279445266998664


In [ ]:
# "compute" intense task

from timeit import timeit
from threading import Thread
from multiprocessing import Process
import asyncio

COUNT = 1000000000


def test(count):
    print("test called")
    for i in range(count):
        1 + 1


def pure():
    test(COUNT)


def with_threading():
    t1 = Thread(target=test, args=(COUNT // 2,))
    t2 = Thread(target=test, args=(COUNT // 2,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()


def with_mp():
    p1 = Process(target=test, args=(COUNT // 2,))
    p2 = Process(target=test, args=(COUNT // 2,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()


async def test_wrapper(count):
    return test(count)


async def async_main():
    asyncio.gather(test_wrapper(COUNT // 2), test_wrapper(COUNT // 2))


def with_asyncio():
    asyncio.run(async_main())


if __name__ == "__main__":
    print("threading", timeit(with_threading, number=1))
    print("mp", timeit(with_mp, number=1))
    print("pure", timeit(pure, number=1))
    print("aio", timeit(with_asyncio, number=1))


# Server/Client Beispiel

# Und Jetzt?

 - `threading` ist super, weil einfach und man kennt das konzept
 - `multiprocessing` auch, aber wenn man mehr Berechnen wll
 - `asyncio` bzw greenlets sind super wenns um viel Netzwerkcode geht


 - [1]: http://axisofeval.blogspot.com/2010/11/numbers-everybody-should-know.html
 - [2]: https://eli.thegreenplace.net/2018/launching-linux-threads-and-processes-with-clone/
 - [3]: https://eli.thegreenplace.net/2018/measuring-context-switching-and-memory-overheads-for-linux-threads/
 - [4]: https://www.youtube.com/watch?v=KXuZi9aeGTw&feature=youtu.be
 